# Library

In [1]:
import pandas as pd
import numpy as np
import glob
import json
from bs4 import BeautifulSoup
from tqdm import tqdm

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Doc2Vec

paper: [Distributed Representations of Sentences and Documents](https://arxiv.org/pdf/1405.4053)

## 문제상황

기존의 방법들, 특히 bag-of-words와 bag-of-n-grams 모델은 아래의 문제를 가지고 있음 <br>

1. 단어 순서 손실: bag-of-words는 단어의 순서를 고려하지 않아, 서로 다른 문장이 동일한 벡터 표현을 가질 수 있음 -> 의미의 구분을 어렵게 만듦

2. 어휘의 의미적 거리 부족: 기존 방법들은 단어들 간의 의미적 유사성을 반영하지 않기 때문에, 의미적으로 가까운 단어와 먼 단어가 동일하게 취급

## Contribution


variable-length text(문장, 단락, 문서 등 다양한 길이의 텍스트)에 대해 고유한 벡터 표현을 생성할 수 있어, 기존의 bag-of-words나 bag-of-n-grams 방식보다 더 유연한 **Paragraph Vector** 제안


## Doc2Vec이란?

<img src="https://i.sstatic.net/t7slV.png" width="700" height="300"/>

### 1.Learning Vector Representation of Words

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcUAewV%2FbtqEp09vwMU%2Fw8XEQM8G0kDwcLyEjs6pSk%2Fimg.png)

모든 단어는 고유한 벡터로 매핑되며, 이 벡터들은 예측을 위해 합쳐지거나 연결. <br>
예측은 softmax를 사용하여 수행되며, 효율적인 학습을 위해 계층적 softmax가 선호 <br>
-> 비슷한 의미의 단어들은 벡터 공간에서 가까운 위치에 매핑

### 2.PV-DM (the Distributed Memory Model of Paragraph Vectors)

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F30I47%2FbtqEq62upsp%2FuKtE1W4eSoJ6jBs41TTRfk%2Fimg.png)

1의 방법에서 paragraph id가 추가된 방법. <br>
각 document에 고유한 벡터 매핑. <br>
sliding window를 통해 현재 단어 예측에 사용할 전후의 단어 개수 설정. <br>
document vector와 선택한 단어 벡터와의 concat 또는 average 후 다음 단어를 예측. <br>
&nbsp;&nbsp;&nbsp;&nbsp; document vector는 해당 document의 모든 문맥에서 공유되지만, 각 문단마다 고유. <br>
&nbsp;&nbsp;&nbsp;&nbsp; 이 과정에서 **단어의 순서**를 고려 (ex) [$\text{word}_{t-1}$, $\text{word}_{t+1}$, document_vector]와 [$\text{word}_{t-1}$, $\text{word}_{t+1}$, document_vector]가 layer를 통과했을 때 학습되는 결과가 달라짐 <br>
&nbsp;&nbsp;&nbsp;&nbsp; concat 방식을 사용하면, 문맥 내 단어의 순서가 명확하게 나타나므로, 모델이 더 풍부한 의미 정보를 학습할 수 있음 <br>
모델 학습 완료된 후 document vector를 feature로 사용하여 기존의 기계 학습 기법에 적용 가능. <br>


### 3.PV-DBOW (the Distributed Bag of Words version of Paragraph Vector)

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FFnx9w%2FbtqEsDLQw44%2FIW4vfgoc9dad5vozNhxAG1%2Fimg.png)

word2vec의 skip-gram과 유사한 방법론. <br>
document vector를 사용하여 랜덤으로 선택된 단어를 예측하는 방식으로 학습. <br>
이 과정에서 단어 순서는 고려되지 않지만, 단락의 의미를 잘 반영하는 벡터 생성. <br>
**메모리 효율성을 제공하는 장점**

## 사용 방법

<br>

> ```python
> model = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.025, workers=8, window=8)
> 
> model.build_vocab(tagged_corpus_list) # vocabulary build
> 
> model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=20) # train
> 
> model.dv.most_similar('[document_name]')  # 유사 문서 검색
>
> model.save('dart.doc2vec')    # save
> model = Doc2Vec.load('/tmp/my_model.doc2vec') # load
> ```

<br>

parameters
- vector_size: vector dimension (몇 차원 벡터로 학습시킬 것인가)
- alpha: learning rate
- min_alpha: 최저 learning rate
- window: window size
- min_count: 학습 시 학습할 단어가 최소 몇 개 등장해야 학습할 것인가
- workers: 학습에 사용할 cpu 수
- dm
    - 0: PV-DBOW
    - 1: PV-DM

# Practice

In [2]:
news = []
file_paths = glob.glob('./data/news_data/**/*.txt', recursive=True)
for file_path in tqdm(file_paths):
    with open(file_path) as file:
        temp = file.read()
        news.append(temp)


def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens


kiwi = Kiwi()
stopwords = Stopwords()

news = pd.DataFrame(news, columns=['contents'])
news = news.drop_duplicates()
news.contents = news.contents.str.replace('\s{1,}', ' ', regex=True)
news['tokens'] = news.contents.apply(lambda x: extract_tokens(x, kiwi, stopwords))

100%|██████████| 1600/1600 [00:00<00:00, 13747.62it/s]


In [6]:
tagged_documents = []
for i, token_list in enumerate(news.tokens):
    tagged_documents.append(
        TaggedDocument(tags=[i], words=token_list)
    )

In [27]:
model = Doc2Vec(
    vector_size=128,
    alpha=1e-3,
    min_alpha=1e-4,
    workers=8,
    window=5,
    dm=0,
    )
model.build_vocab(tagged_documents)
model.train(
    tagged_documents,
    total_examples=model.corpus_count,
    epochs=20,
)

In [29]:
model.dv.most_similar(15)

[(838, 0.3954414427280426),
 (1416, 0.3784639835357666),
 (947, 0.3239007890224457),
 (1330, 0.322730153799057),
 (653, 0.3110679090023041),
 (1549, 0.31089287996292114),
 (793, 0.30750396847724915),
 (869, 0.3064292371273041),
 (55, 0.2939872741699219),
 (1552, 0.2923944592475891)]

In [30]:
news.iloc[[15, 838]]

,contents,tokens
15,징검다리 연휴 나들이…전국 고속道 심한 정체 서울~부산 5시간30분 오후 9~10시...,"[징검다리, 연휴, 나들이, 전국, 고속, 정체, 서울, 부산, 오후, 해소, 서울..."
854,"靑, NSC 개최…""북·미 사이에 입장 차이…중재자 역할할 것""(종합) [아시아경제...","[개최, 사이, 입장, 차이, 중재자, 역할, 종합, 아시아, 경제, 진영, 기자,..."


## 사업의 개요

In [34]:
data = pd.read_pickle('./data/crawl_data.pickle')
data = data.query('dates == "2022.03"')

data['business_info'] = data['사업의 개요'].apply(lambda x: BeautifulSoup(x, 'lxml').text)
data.business_info = data.business_info.str.replace('\s+', ' ', regex=True)
data.business_info = data.business_info.str.replace('\d?\.? ? 사업의 개요 ?', '', regex=True)

def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

kiwi = Kiwi()
kiwi.add_user_word('바이오시밀러', 'NNG')
kiwi.add_user_word('웅진씽크빅', 'NNP')

stopwords = Stopwords()
stopwords.add(('당사', 'NNG'))
stopwords.add(('산업', 'NNG'))
stopwords.add(('특성', 'NNG'))

data['tokens'] = data.business_info.apply(lambda x: extract_tokens(x, kiwi, stopwords))

In [38]:
tagged_document = []
for row in data.itertuples():
    tagged_documents.append(TaggedDocument(tags=[row.corp_name], words=row.tokens))

model = Doc2Vec(
    vector_size=128,
    alpha=1e-2,
    min_alpha=1e-3,
    workers=8,
    window=8,
    dm=0,
    )
model.build_vocab(tagged_documents)
model.train(
    tagged_documents,
    total_examples=model.corpus_count,
    epochs=20,
)

In [39]:
model.dv.most_similar('삼성전자')

[('금호타이어', 0.7989670634269714),
 ('휴니드테크놀러지스', 0.7830410599708557),
 ('삼성전기', 0.7797526121139526),
 ('삼영전자공업', 0.7634993195533752),
 ('SK하이닉스', 0.762822687625885),
 ('화천기공', 0.7555910348892212),
 ('KPX케미칼', 0.7461060881614685),
 ('LX세미콘', 0.7452780604362488),
 ('솔루엠', 0.7375050783157349),
 ('삼익악기', 0.7337893843650818)]

## 특허

In [ ]:
data = []
file_paths = glob.glob(r'.\data\patent\A_농업_임업및어업_01_03\01_농업\**\*.json')

for file_path in file_paths:
    with open(file_path) as file:
        datum = json.load(file).get('dataset')
    data.extend(datum)

In [38]:
# task1: 곡물및기타식량작물재배업
## tag: agriculture1, agriculture2, ...
# task2: 농업 전체

claims = [datum.get('claims') for datum in data]

In [39]:
kiwi = Kiwi()
stopwords = Stopwords()


def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # string이 빈 값이면 early return
    if not string:
        return []

    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

In [42]:
tokens = pd.Series(claims).apply(lambda x: extract_tokens(x, kiwi, stopwords))

tagged_documents = []
for i, token in enumerate(tokens):
    tagged_documents.append(TaggedDocument(tags=[f'agriculture{i+1}'], words=token))

In [24]:
model = Doc2Vec(
    vector_size=128,
    alpha=1e-3,
    min_alpha=1e-3,
    workers=8,
    window=5,
    dm_concat=1,
    dm=0,
    )
model.build_vocab(tagged_documents)
model.train(
    tagged_documents,
    total_examples=model.corpus_count,
    epochs=15,
)

In [25]:
model.dv.most_similar('agriculture1')

[('agriculture766', 0.625105082988739),
 ('agriculture219', 0.6058903336524963),
 ('agriculture580', 0.6004685163497925),
 ('agriculture237', 0.592998206615448),
 ('agriculture716', 0.5876607298851013),
 ('agriculture494', 0.587647557258606),
 ('agriculture496', 0.5862235426902771),
 ('agriculture492', 0.5852714776992798),
 ('agriculture498', 0.5843524932861328),
 ('agriculture300', 0.583195686340332)]